In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mabdullahsajid/celeba-spoofing")

print("Path to dataset files:", path)

100%|███████████████████████████████████████████████████████████████████████████| 71.9G/71.9G [2:14:24<00:00, 9.58MB/s]

Extracting files...


Path to dataset files: C:\Users\mujta\.cache\kagglehub\datasets\mabdullahsajid\celeba-spoofing\versions\1


In [1]:
import kagglehub

paths = {
    "celeba_spoof": kagglehub.dataset_download("mabdullahsajid/celeba-spoofing"),
    "casia_fasd": kagglehub.dataset_download("immada/casia-fasd"),
    "replay_attack": kagglehub.dataset_download("axondata/replay-attack-mobile-dataset-ibeta-1"),
    "siw": kagglehub.dataset_download("kokomi3012/siw-dataset"),
}

for name, path in paths.items():
    print(f"\n📁 Dataset: {name}")
    print(f"Path: {path}\n")

import os

def list_dir_structure(base_path, max_depth=3, prefix=""):
    if max_depth < 1:
        return
    try:
        for entry in os.listdir(base_path):
            full_path = os.path.join(base_path, entry)
            print(prefix + "├── " + entry)
            if os.path.isdir(full_path):
                list_dir_structure(full_path, max_depth - 1, prefix + "│   ")
    except Exception as e:
        print(f"⚠️ Error reading {base_path}: {e}")

for name, path in paths.items():
    print(f"\n🔍 Directory structure for {name}:\n")
    list_dir_structure(path, max_depth=3)
    print("\n" + "="*80)


📁 Dataset: celeba_spoof
Path: C:\Users\mujta\.cache\kagglehub\datasets\mabdullahsajid\celeba-spoofing\versions\1


📁 Dataset: casia_fasd
Path: C:\Users\mujta\.cache\kagglehub\datasets\immada\casia-fasd\versions\1


📁 Dataset: replay_attack
Path: C:\Users\mujta\.cache\kagglehub\datasets\axondata\replay-attack-mobile-dataset-ibeta-1\versions\4


📁 Dataset: siw
Path: C:\Users\mujta\.cache\kagglehub\datasets\kokomi3012\siw-dataset\versions\1


🔍 Directory structure for celeba_spoof:

├── CelebA_Spoof
│   ├── Data
│   │   ├── test
│   │   ├── train
│   ├── metas
│   │   ├── intra_test
│   │   ├── protocol1
│   │   ├── protocol2
│   ├── README


🔍 Directory structure for casia_fasd:

├── casia-fasd
│   ├── test
│   │   ├── live
│   │   ├── spoof
│   ├── train
│   │   ├── live
│   │   ├── spoof


🔍 Directory structure for replay_attack:

├── Axon Labs Replay mobile sample
│   ├── Galaxy_a54-Honor70
│   │   ├── 20241126_024625.mp4
│   │   ├── 20241126_024646.mp4
│   │   ├── 20241126_024707.mp

In [4]:
import shutil, os

# CASIA-FASD paths
src_train = r"C:\Users\mujta\.cache\kagglehub\datasets\immada\casia-fasd\versions\1\casia-fasd\train"
src_test = r"C:\Users\mujta\.cache\kagglehub\datasets\immada\casia-fasd\versions\1\casia-fasd\test"
dst_root = r"C:\Users\mujta\data"

# Create destination directories
os.makedirs(f"{dst_root}/train/real", exist_ok=True)
os.makedirs(f"{dst_root}/train/spoof", exist_ok=True)
os.makedirs(f"{dst_root}/test/real", exist_ok=True)
os.makedirs(f"{dst_root}/test/spoof", exist_ok=True)

# ✅ Correct mapping: "live" → "real"
shutil.copytree(f"{src_train}/live", f"{dst_root}/train/real", dirs_exist_ok=True)
shutil.copytree(f"{src_train}/spoof", f"{dst_root}/train/spoof", dirs_exist_ok=True)
shutil.copytree(f"{src_test}/live", f"{dst_root}/test/real", dirs_exist_ok=True)
shutil.copytree(f"{src_test}/spoof", f"{dst_root}/test/spoof", dirs_exist_ok=True)

print("✅ CASIA-FASD merged successfully!")

✅ CASIA-FASD merged successfully!


In [5]:
src_siw = r"C:\Users\mujta\.cache\kagglehub\datasets\kokomi3012\siw-dataset\versions\1\SiW"
for split in ["train", "val", "test"]:
    for category in ["real", "spoof"]:
        shutil.copytree(f"{src_siw}/{split}/{category}", f"{dst_root}/{split}/{category}", dirs_exist_ok=True)

In [6]:
import cv2, random

src_replay = r"C:\Users\mujta\.cache\kagglehub\datasets\axondata\replay-attack-mobile-dataset-ibeta-1\versions\4\Axon Labs Replay mobile sample"
dst_spoof = f"{dst_root}/train/spoof"

for folder in os.listdir(src_replay):
    for file in os.listdir(os.path.join(src_replay, folder)):
        if file.endswith(".mp4"):
            video_path = os.path.join(src_replay, folder, file)
            cap = cv2.VideoCapture(video_path)
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            for i in random.sample(range(total_frames), min(10, total_frames)):
                cap.set(cv2.CAP_PROP_POS_FRAMES, i)
                ret, frame = cap.read()
                if ret:
                    filename = f"{os.path.splitext(file)[0]}_{i}.jpg"
                    cv2.imwrite(os.path.join(dst_spoof, filename), frame)
            cap.release()

In [5]:
import os, json, shutil
from tqdm import tqdm

# Base paths
base = r"C:\Users\mujta\.cache\kagglehub\datasets\mabdullahsajid\celeba-spoofing\versions\1\CelebA_Spoof"
dst_root = r"C:\Users\mujta\data"

# Create destination folders
for split in ["train", "test"]:
    os.makedirs(f"{dst_root}/{split}/real", exist_ok=True)
    os.makedirs(f"{dst_root}/{split}/spoof", exist_ok=True)

# Load metadata
with open(f"{base}/metas/protocol1/train_label.json", "r") as f:
    meta_train = json.load(f)

with open(f"{base}/metas/protocol1/test_label.json", "r") as f:
    meta_test = json.load(f)


def copy_split(meta, split):
    for img_rel_path, label_info in tqdm(meta.items(), total=len(meta), desc=f"Copying {split}"):
        # Determine full source path
        src = os.path.join(base, img_rel_path)

        # CelebA Spoof labels are stored as lists, where last element is label
        # Example: [0, 1, 0, 1] -> spoof label is last (label_info[-1])
        if isinstance(label_info, list):
            spoof_label = label_info[-1]
        elif isinstance(label_info, dict) and "spoof_label" in label_info:
            spoof_label = label_info["spoof_label"]
        else:
            continue  # unknown format, skip

        # Destination folder
        dst = f"{dst_root}/{split}/spoof" if spoof_label == 1 else f"{dst_root}/{split}/real"

        try:
            shutil.copy(src, dst)
        except FileNotFoundError:
            continue


# Run for both splits
copy_split(meta_train, "train")
copy_split(meta_test, "test")

print("✅ CelebA-Spoof dataset merged successfully!")

Copying test: 100%|██████████████████████████████████████████████████████████████| 25758/25758 [07:45<00:00, 55.36it/s]

✅ CelebA-Spoof dataset merged successfully!


In [12]:
dst_root = r"C:\Users\mujta\data"

for split in ["train", "val", "test"]:
    for cls in ["real", "spoof"]:
        count = len(os.listdir(f"{dst_root}/{split}/{cls}"))
        print(f"{split}/{cls}: {count}")

train/real: 106614
train/spoof: 201883
val/real: 600
val/spoof: 150
test/real: 19639
test/spoof: 72655


In [13]:
import os, random, shutil
from tqdm import tqdm

def balance_dataset(root):
    for split in ["train", "val", "test"]:
        real_dir = os.path.join(root, split, "real")
        spoof_dir = os.path.join(root, split, "spoof")

        real_imgs = os.listdir(real_dir)
        spoof_imgs = os.listdir(spoof_dir)

        n_real = len(real_imgs)
        n_spoof = len(spoof_imgs)

        # If spoof > real → reduce spoof
        if n_spoof > n_real:
            spoof_imgs = random.sample(spoof_imgs, n_real)
        # If real > spoof → reduce real
        elif n_real > n_spoof:
            real_imgs = random.sample(real_imgs, n_spoof)

        # Move balanced images into new folder
        balanced_root = root + "_balanced"
        os.makedirs(os.path.join(balanced_root, split, "real"), exist_ok=True)
        os.makedirs(os.path.join(balanced_root, split, "spoof"), exist_ok=True)

        for img in tqdm(real_imgs, desc=f"{split}/real"):
            shutil.copy(os.path.join(real_dir, img), os.path.join(balanced_root, split, "real", img))
        for img in tqdm(spoof_imgs, desc=f"{split}/spoof"):
            shutil.copy(os.path.join(spoof_dir, img), os.path.join(balanced_root, split, "spoof", img))

    print("✅ Dataset balanced and saved to:", balanced_root)

balance_dataset(dst_root)

test/spoof: 100%|████████████████████████████████████████████████████████████████| 19639/19639 [04:23<00:00, 74.66it/s]

✅ Dataset balanced and saved to: C:\Users\mujta\data_balanced


In [14]:
import os
import cv2
import torch
from facenet_pytorch import MTCNN
from tqdm import tqdm
from PIL import Image
import numpy as np

# Paths
src_root = r"C:\Users\mujta\data_balanced"
dst_root = r"C:\Users\mujta\data_faces"

# Initialize MTCNN (GPU if available)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(image_size=224, margin=20, keep_all=False, post_process=False, device=device)

# Create destination dirs
for split in ["train", "val", "test"]:
    for label in ["real", "spoof"]:
        os.makedirs(f"{dst_root}/{split}/{label}", exist_ok=True)


def crop_faces(split):
    for label in ["real", "spoof"]:
        src_dir = f"{src_root}/{split}/{label}"
        dst_dir = f"{dst_root}/{split}/{label}"

        for img_name in tqdm(os.listdir(src_dir), desc=f"{split}/{label}"):
            src_path = os.path.join(src_dir, img_name)
            dst_path = os.path.join(dst_dir, img_name)
            try:
                img = cv2.imread(src_path)
                if img is None:
                    continue

                # Convert BGR → RGB for MTCNN
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img_pil = Image.fromarray(img_rgb)

                # Detect and crop
                face = mtcnn(img_pil)

                if face is not None:
                    # Convert tensor → NumPy image
                    face = face.permute(1, 2, 0).byte().cpu().numpy()
                    face_bgr = cv2.cvtColor(face, cv2.COLOR_RGB2BGR)
                    cv2.imwrite(dst_path, face_bgr)

            except Exception as e:
                continue


# Process splits
crop_faces("train")
crop_faces("val")
crop_faces("test")

print("✅ Faces cropped and aligned successfully!")

test/spoof: 100%|████████████████████████████████████████████████████████████████| 19639/19639 [25:25<00:00, 12.87it/s]

✅ Faces cropped and aligned successfully!


In [15]:
for split in ["train", "val", "test"]:
    for cls in ["real", "spoof"]:
        count = len(os.listdir(f"{dst_root}/{split}/{cls}"))
        print(f"{split}/{cls}: {count}")

train/real: 106406
train/spoof: 103599
val/real: 150
val/spoof: 150
test/real: 19618
test/spoof: 19393


In [16]:
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([transforms.GaussianBlur(3)], p=0.3),
    transforms.RandomErasing(p=0.2),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

In [ ]:
# Step 3: Train custom CNN on data_faces
import os
import copy
import time
from tqdm import tqdm

import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# --------- Config ----------
DATA_ROOT = r"C:\Users\mujta\data_faces"   # <--- your balanced dataset root
NUM_CLASSES = 2
BATCH_SIZE = 64
IMG_SIZE = 224
NUM_EPOCHS = 10
LR = 1e-3
PATIENCE = 5       # early stopping patience
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_SAVE_PATH = "best_model.pth"
# --------------------------

# -------- Transforms ----------
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.15),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.2),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])
# ------------------------------

# -------- Datasets / Loaders ----------
train_dataset = datasets.ImageFolder(os.path.join(DATA_ROOT, "train"), transform=train_transforms)
val_dataset   = datasets.ImageFolder(os.path.join(DATA_ROOT, "val"), transform=val_transforms)
test_dataset  = datasets.ImageFolder(os.path.join(DATA_ROOT, "test"), transform=val_transforms)

# sanity check class mapping
print("Class to idx:", train_dataset.class_to_idx)  # note: e.g., {'real':0, 'spoof':1}

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
# -------------------------------------

# -------- Model (Simple CNN) ----------
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.AdaptiveAvgPool2d(1)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = SimpleCNN(num_classes=NUM_CLASSES).to(DEVICE)
# -------------------------------------

# Loss, optimizer, scheduler
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

# Utility: compute metrics and FAS-specific metrics
def compute_metrics(y_true, y_pred, class_to_idx):
    # y_true/y_pred are 1D numpy arrays of labels (0 real, 1 spoof) — check class mapping
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    tn, fp, fn, tp = cm.ravel()
    acc = (tp + tn) / cm.sum() if cm.sum() > 0 else 0.0
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    # For FAS metrics (assume label 1 == spoof, 0 == real)
    # APCER: proportion of attacks (spoof) classified as bona fide (real) => fp_on_spoof / total_spoof
    # BPCER: proportion of bona fide (real) classified as attack (spoof) => fn_on_real / total_real
    total_spoof = np.sum(y_true == 1)
    total_real  = np.sum(y_true == 0)
    # attacks classified as bona fide: predicted 0 but true=1 -> count where (true==1 & pred==0)
    attacks_misclassified_as_real = np.sum((y_true == 1) & (y_pred == 0))
    bona_misclassified_as_attack = np.sum((y_true == 0) & (y_pred == 1))
    apcer = attacks_misclassified_as_real / total_spoof if total_spoof>0 else 0.0
    bpcer = bona_misclassified_as_attack / total_real if total_real>0 else 0.0
    acer = 0.5 * (apcer + bpcer)

    return {
        "confusion_matrix": cm,
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "APCER": apcer,
        "BPCER": bpcer,
        "ACER": acer
    }

# Training / validation loop
best_val_loss = float('inf')
best_model_wts = copy.deepcopy(model.state_dict())
epochs_no_improve = 0

for epoch in range(1, NUM_EPOCHS+1):
    t0 = time.time()
    # --- Train ---
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch} - Train"):
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        preds = logits.argmax(dim=1).detach().cpu().numpy()
        all_preds.append(preds)
        all_labels.append(labels.detach().cpu().numpy())

    train_loss = running_loss / len(train_dataset)
    train_preds = np.concatenate(all_preds)
    train_labels = np.concatenate(all_labels)
    train_metrics = compute_metrics(train_labels, train_preds, train_dataset.class_to_idx)

    # --- Validate ---
    model.eval()
    val_loss = 0.0
    all_val_preds = []
    all_val_labels = []
    with torch.no_grad():
        for imgs, labels in tqdm(val_loader, desc=f"Epoch {epoch} - Val"):
            imgs = imgs.to(DEVICE)
            labels = labels.to(DEVICE)
            logits = model(imgs)
            loss = criterion(logits, labels)
            val_loss += loss.item() * imgs.size(0)
            preds = logits.argmax(dim=1).cpu().numpy()
            all_val_preds.append(preds)
            all_val_labels.append(labels.cpu().numpy())

    val_loss = val_loss / len(val_dataset)
    val_preds = np.concatenate(all_val_preds) if all_val_preds else np.array([])
    val_labels = np.concatenate(all_val_labels) if all_val_labels else np.array([])
    val_metrics = compute_metrics(val_labels, val_preds, val_dataset.class_to_idx) if val_preds.size else {}

    # Scheduler step
    scheduler.step(val_loss)

    # Print epoch summary
    elapsed = time.time() - t0
    print(f"\nEpoch {epoch} | time: {elapsed:.1f}s | train_loss: {train_loss:.4f} | val_loss: {val_loss:.4f}")
    print("Train ACC/F1/APCER/ACER:", train_metrics['accuracy'], train_metrics['f1'], train_metrics['APCER'], train_metrics['ACER'])
    print("Val   ACC/F1/APCER/ACER:", val_metrics.get('accuracy'), val_metrics.get('f1'), val_metrics.get('APCER'), val_metrics.get('ACER'))

    # Save best
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss
        }, MODEL_SAVE_PATH)
        print("Saved best model.")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f"No improvement for {epochs_no_improve} epochs.")

    if epochs_no_improve >= PATIENCE:
        print("Early stopping triggered.")
        break

# Load best weights
model.load_state_dict(best_model_wts)
print("Training finished. Best val loss:", best_val_loss)

# -------- Evaluate on test set ----------
model.eval()
all_test_preds = []
all_test_labels = []
with torch.no_grad():
    for imgs, labels in tqdm(test_loader, desc="Testing"):
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)
        logits = model(imgs)
        preds = logits.argmax(dim=1).cpu().numpy()
        all_test_preds.append(preds)
        all_test_labels.append(labels.cpu().numpy())

test_preds = np.concatenate(all_test_preds)
test_labels = np.concatenate(all_test_labels)
test_metrics = compute_metrics(test_labels, test_preds, train_dataset.class_to_idx)

print("\n--- Test Metrics ---")
print("Confusion Matrix:\n", test_metrics["confusion_matrix"])
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"F1: {test_metrics['f1']:.4f}")
print(f"APCER: {test_metrics['APCER']:.4f}")
print(f"BPCER: {test_metrics['BPCER']:.4f}")
print(f"ACER: {test_metrics['ACER']:.4f}")

# Save final model (weights only)
torch.save(model.state_dict(), "final_model_weights.pth")
print("Saved final_model_weights.pth")

Class to idx: {'real': 0, 'spoof': 1}


C:\Users\mujta\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 1 - Train: 100%|███████████████████████████████████████████████████████████| 3282/3282 [3:46:02<00:00,  4.13s/it]
C:\Users\mujta\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 1 - Val: 100%|█████████████████████████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.55s/it]



Epoch 1 | time: 13570.2s | train_loss: 0.3640 | val_loss: 0.3959
Train ACC/F1/APCER/ACER: 0.8398942882312326 0.8387053445075003 0.15619841890365738 0.16005417439741448
Val   ACC/F1/APCER/ACER: 0.8266666666666667 0.7952755905511811 0.32666666666666666 0.17333333333333334
Saved best model.


C:\Users\mujta\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 2 - Train:  24%|██████████████                                              | 772/3282 [43:41<2:15:23,  3.24s/it]